# Twitter Klassifizierung nach TUWien (GermEval2018)
Das Training und die eigentliche Klassifizierung wird im Python File 'Twitter_Sent_Analyse_TUWien_auto' durchgeführt. In diesem File findet sich:
* Die Verbesserung und Analyse der Trainingsdaten und Ergebnisse
* Zusammenstellung des Endgültigen Trainingssatzes und Evaluierung (Accuracy, Precission, Recall, F1)

In [1]:
Path = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/01_Twitter'

### Verhältnis von OFFENSE Tweets zu Rest - Erstellung eigener TrainingsDaten
Hinweis: manche der hier verwendeten .txt files wurden der Übersicht halber gelöscht. Der folgende Code dient daher primär zum nachvollziehen, als zum direkten nachbilden.

In [11]:
import pickle, gensim, numpy as np
import csv
import pandas as pd
import datetime

## Lade Datensätze und erstelle DataFrames
from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken
more_train = get_train_data(Path + '/Evaluierung_Sampling/more_train.csv')
train = {'Text': more_train[0], 'Label': more_train[1]} 
df_more_train = pd.DataFrame(train)

GermEval_train = get_train_data(Path + '/Evaluierung_Sampling/germeval2018.training_all_8541.txt')
train_GermEval = {'Text': GermEval_train[0], 'Label': GermEval_train[1]} 
df_GermEval_train = pd.DataFrame(train_GermEval)

compare_old_new = pickle.load(open(Path+"/Evaluierung_Sampling/compare_old_new_label.p", "rb" ))
#old_all = pickle.load(open(Path + 'sent_df_Wien', "rb" ))

In [3]:
## Vergleich der Verhältnisse von OFFENSE Tweets zum Rest
perc_OFFENSE_train = (len(df_more_train[df_more_train['Label'] == 'OFFENSE']) / len(df_more_train)) * 100
perc_OFFENSE_GermEval = (len(df_GermEval_train[df_GermEval_train['Label'] == 'OFFENSE']) / len(df_GermEval_train)) * 100
perc_OFFENSE_all = (len(old_all[old_all['predict'] == 'OFFENSE']) / len(old_all['predict'])) * 100

print('Im aktuellen more_training file liegt der Anteil von OFFENSE Tweets bei: ' + str(perc_OFFENSE_train))
print('Im GermEval Trainings Satz liegt der Anteil von OFFENSE Tweets bei: ' + str(perc_OFFENSE_GermEval))
print('Im anfänglichen gesamtDatensatz lag nach dem Labeling mit GermEval der Anteil von OFFENSE Tweets bei: ' + str(perc_OFFENSE_all))

Im aktuellen more_training file liegt der Anteil von OFFENSE Tweets bei: 50.706214689265536
Im GermEval Trainings Satz liegt der Anteil von OFFENSE Tweets bei: 33.8367872614448
Im anfänglichen gesamtDatensatz lag nach dem Labeling mit GermEval der Anteil von OFFENSE Tweets bei: 6.96534218199701


### Laden der gelabelten Datensätze und zusammenfügen

In [4]:
PICKLE_FOLDER_PATH_old = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/pickle_TU_Wien/compare/'
PICKLE_FOLDER_PATH_new = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/pickle_TU_Wien/'

# Die csv files 'old' und 'new' werden im jeweiligen Verzeichnis erstellt:

with open(PICKLE_FOLDER_PATH_old + 'old.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        old = row

with open(PICKLE_FOLDER_PATH_new + 'new.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        new = row

        
# Jeder gelabelte DataFrame wird aufgerufen und in einem Dictionary gespeichert
Twitter_test_new = {}
Twitter_test_old = {}

for file in new:
    Twitter_test_new[file] = pickle.load(open(PICKLE_FOLDER_PATH_new + file, "rb" ))

for file in old:
    Twitter_test_old[file] = pickle.load(open(PICKLE_FOLDER_PATH_old + file, "rb" ))


In [5]:
# Die DataFrames werden aus den Dictionarys erstellt

Twitter_comp_new = pd.concat(Twitter_test_new, axis=0)
Twitter_comp_old = pd.concat(Twitter_test_old, axis=0)

Twitter_comp_old.loc[:,('Woche')] = Twitter_comp_old.created_at_datetime.dt.strftime('%y-w%U')
# Nur die Teile des alten DataFrames werden betrachtet, die im gleichen Zeitraum sind wie der neue
Twitter_comp_old = Twitter_comp_old.loc[(Twitter_comp_old['Woche'] == '20-w03') |
                                       (Twitter_comp_old['Woche'] == '20-w06')|
                                       (Twitter_comp_old['Woche'] == '20-w08')|
                                       (Twitter_comp_old['Woche'] == '20-w09')|
                                       (Twitter_comp_old['Woche'] == '20-w15')|
                                       (Twitter_comp_old['Woche'] == '20-w19')|
                                       (Twitter_comp_old['Woche'] == '20-w23')|
                                       (Twitter_comp_old['Woche'] == '20-w25')|
                                       (Twitter_comp_old['Woche'] == '20-w27')]

### Erstelle DataFrame zum Vergleich der "alten" und "neuen" gelabelten Sätze

In [6]:
twit_comp_new = []
twit_comp_old = []
twit_text_new = []
twit_text_old = []

for i in range(0, len(Twitter_comp_new)):
    if Twitter_comp_new['full_text_processed'][i] in set(Twitter_comp_old['full_text_processed']):
        twit_comp_new.append(Twitter_comp_new['predict'][i])
        twit_comp_old.append(Twitter_comp_old[Twitter_comp_old['full_text_processed'] == Twitter_comp_new['full_text_processed'][i]]['predict'].values[0])
        twit_text_new.append(Twitter_comp_new['full_text_processed'][i])
        twit_text_old.append(Twitter_comp_old[Twitter_comp_old['full_text_processed'] == Twitter_comp_new['full_text_processed'][i]]['full_text_processed'].values[0])
    else:
        print('Keine Überlappung bei: ' + str(i))

Keine Überlappung bei: 3028
Keine Überlappung bei: 3029
Keine Überlappung bei: 3030
Keine Überlappung bei: 3031
Keine Überlappung bei: 3032
Keine Überlappung bei: 3033
Keine Überlappung bei: 3034
Keine Überlappung bei: 3035
Keine Überlappung bei: 3036
Keine Überlappung bei: 3037
Keine Überlappung bei: 3038
Keine Überlappung bei: 3039
Keine Überlappung bei: 3040
Keine Überlappung bei: 3041
Keine Überlappung bei: 3042
Keine Überlappung bei: 3045
Keine Überlappung bei: 3046
Keine Überlappung bei: 3047
Keine Überlappung bei: 3048
Keine Überlappung bei: 3049
Keine Überlappung bei: 3050
Keine Überlappung bei: 3051
Keine Überlappung bei: 3052
Keine Überlappung bei: 3053
Keine Überlappung bei: 3054
Keine Überlappung bei: 3058
Keine Überlappung bei: 3059
Keine Überlappung bei: 3060
Keine Überlappung bei: 3061
Keine Überlappung bei: 3062
Keine Überlappung bei: 3063
Keine Überlappung bei: 3064
Keine Überlappung bei: 3065
Keine Überlappung bei: 3066
Keine Überlappung bei: 3395
Keine Überlappung be

Keine Überlappung bei: 3661
Keine Überlappung bei: 3662
Keine Überlappung bei: 3663
Keine Überlappung bei: 3664
Keine Überlappung bei: 3665
Keine Überlappung bei: 3666
Keine Überlappung bei: 3667
Keine Überlappung bei: 3668
Keine Überlappung bei: 3669
Keine Überlappung bei: 3670
Keine Überlappung bei: 3671
Keine Überlappung bei: 3672
Keine Überlappung bei: 3673
Keine Überlappung bei: 3674
Keine Überlappung bei: 3675
Keine Überlappung bei: 3676
Keine Überlappung bei: 3677
Keine Überlappung bei: 3678
Keine Überlappung bei: 3679
Keine Überlappung bei: 3680
Keine Überlappung bei: 3681
Keine Überlappung bei: 3682
Keine Überlappung bei: 3683
Keine Überlappung bei: 3684
Keine Überlappung bei: 3685
Keine Überlappung bei: 3686
Keine Überlappung bei: 3687
Keine Überlappung bei: 3688
Keine Überlappung bei: 3689
Keine Überlappung bei: 3690
Keine Überlappung bei: 3691
Keine Überlappung bei: 3692
Keine Überlappung bei: 3693
Keine Überlappung bei: 3694
Keine Überlappung bei: 3695
Keine Überlappung be

Keine Überlappung bei: 3963
Keine Überlappung bei: 3964
Keine Überlappung bei: 3965
Keine Überlappung bei: 3966
Keine Überlappung bei: 3967
Keine Überlappung bei: 3968
Keine Überlappung bei: 3969
Keine Überlappung bei: 3970
Keine Überlappung bei: 3971
Keine Überlappung bei: 3972
Keine Überlappung bei: 3973
Keine Überlappung bei: 3974
Keine Überlappung bei: 3975
Keine Überlappung bei: 3976
Keine Überlappung bei: 3977
Keine Überlappung bei: 3978
Keine Überlappung bei: 3979
Keine Überlappung bei: 3980
Keine Überlappung bei: 3981
Keine Überlappung bei: 3982
Keine Überlappung bei: 3983
Keine Überlappung bei: 3984
Keine Überlappung bei: 3985
Keine Überlappung bei: 3986
Keine Überlappung bei: 3987
Keine Überlappung bei: 3988
Keine Überlappung bei: 3989
Keine Überlappung bei: 3990
Keine Überlappung bei: 3991
Keine Überlappung bei: 3992
Keine Überlappung bei: 3993
Keine Überlappung bei: 3995
Keine Überlappung bei: 3996
Keine Überlappung bei: 3997
Keine Überlappung bei: 3998
Keine Überlappung be

Keine Überlappung bei: 4273
Keine Überlappung bei: 4274
Keine Überlappung bei: 4275
Keine Überlappung bei: 4276
Keine Überlappung bei: 4277
Keine Überlappung bei: 4278
Keine Überlappung bei: 4279
Keine Überlappung bei: 4280
Keine Überlappung bei: 4281
Keine Überlappung bei: 4282
Keine Überlappung bei: 4283
Keine Überlappung bei: 4284
Keine Überlappung bei: 4285
Keine Überlappung bei: 4286
Keine Überlappung bei: 4287
Keine Überlappung bei: 4288
Keine Überlappung bei: 4289
Keine Überlappung bei: 4290
Keine Überlappung bei: 4291
Keine Überlappung bei: 4292
Keine Überlappung bei: 4293
Keine Überlappung bei: 4294
Keine Überlappung bei: 4295
Keine Überlappung bei: 4296
Keine Überlappung bei: 4297
Keine Überlappung bei: 4298
Keine Überlappung bei: 4299
Keine Überlappung bei: 4300
Keine Überlappung bei: 4301
Keine Überlappung bei: 4302
Keine Überlappung bei: 4303
Keine Überlappung bei: 4304
Keine Überlappung bei: 4305
Keine Überlappung bei: 4306
Keine Überlappung bei: 4307
Keine Überlappung be

Keine Überlappung bei: 4574
Keine Überlappung bei: 4575
Keine Überlappung bei: 4576
Keine Überlappung bei: 4577
Keine Überlappung bei: 4578
Keine Überlappung bei: 4579
Keine Überlappung bei: 4580
Keine Überlappung bei: 4581
Keine Überlappung bei: 4582
Keine Überlappung bei: 4583
Keine Überlappung bei: 4584
Keine Überlappung bei: 4585
Keine Überlappung bei: 4586
Keine Überlappung bei: 4587
Keine Überlappung bei: 4588
Keine Überlappung bei: 4589
Keine Überlappung bei: 4590
Keine Überlappung bei: 4591
Keine Überlappung bei: 4592
Keine Überlappung bei: 4593
Keine Überlappung bei: 4594
Keine Überlappung bei: 4595
Keine Überlappung bei: 4596
Keine Überlappung bei: 4597
Keine Überlappung bei: 4598
Keine Überlappung bei: 4599
Keine Überlappung bei: 4600
Keine Überlappung bei: 4601
Keine Überlappung bei: 4602
Keine Überlappung bei: 4603
Keine Überlappung bei: 4604
Keine Überlappung bei: 4605
Keine Überlappung bei: 4606
Keine Überlappung bei: 4607
Keine Überlappung bei: 4608
Keine Überlappung be

...

Keine Überlappung bei: 63075
Keine Überlappung bei: 63076
Keine Überlappung bei: 63077
Keine Überlappung bei: 63078
Keine Überlappung bei: 63079
Keine Überlappung bei: 63080
Keine Überlappung bei: 63081
Keine Überlappung bei: 63082
Keine Überlappung bei: 63083
Keine Überlappung bei: 63084
Keine Überlappung bei: 63085
Keine Überlappung bei: 63086
Keine Überlappung bei: 63087
Keine Überlappung bei: 63088
Keine Überlappung bei: 63089
Keine Überlappung bei: 63090
Keine Überlappung bei: 63091
Keine Überlappung bei: 63092
Keine Überlappung bei: 63093
Keine Überlappung bei: 63094
Keine Überlappung bei: 63095
Keine Überlappung bei: 63096
Keine Überlappung bei: 63097
Keine Überlappung bei: 63098
Keine Überlappung bei: 63099
Keine Überlappung bei: 63100
Keine Überlappung bei: 63101
Keine Überlappung bei: 63102
Keine Überlappung bei: 63103
Keine Überlappung bei: 63104
Keine Überlappung bei: 63105
Keine Überlappung bei: 63106
Keine Überlappung bei: 63107
Keine Überlappung bei: 63108
Keine Überlapp

Keine Überlappung bei: 63370
Keine Überlappung bei: 63371
Keine Überlappung bei: 63372
Keine Überlappung bei: 63373
Keine Überlappung bei: 63374
Keine Überlappung bei: 63375
Keine Überlappung bei: 63376
Keine Überlappung bei: 63377
Keine Überlappung bei: 63378
Keine Überlappung bei: 63379
Keine Überlappung bei: 63380
Keine Überlappung bei: 63381
Keine Überlappung bei: 63382
Keine Überlappung bei: 63383
Keine Überlappung bei: 63384
Keine Überlappung bei: 63385
Keine Überlappung bei: 63386
Keine Überlappung bei: 63387
Keine Überlappung bei: 63388
Keine Überlappung bei: 63389
Keine Überlappung bei: 63390
Keine Überlappung bei: 63391
Keine Überlappung bei: 63392
Keine Überlappung bei: 63393
Keine Überlappung bei: 63394
Keine Überlappung bei: 63395
Keine Überlappung bei: 63396
Keine Überlappung bei: 63397
Keine Überlappung bei: 63398
Keine Überlappung bei: 63399
Keine Überlappung bei: 63400
Keine Überlappung bei: 63401
Keine Überlappung bei: 63402
Keine Überlappung bei: 63403
Keine Überlapp

Keine Überlappung bei: 63663
Keine Überlappung bei: 63664
Keine Überlappung bei: 63665
Keine Überlappung bei: 63667
Keine Überlappung bei: 63668
Keine Überlappung bei: 63669
Keine Überlappung bei: 63670
Keine Überlappung bei: 63671
Keine Überlappung bei: 63672
Keine Überlappung bei: 63673
Keine Überlappung bei: 63674
Keine Überlappung bei: 63675
Keine Überlappung bei: 63676
Keine Überlappung bei: 63677
Keine Überlappung bei: 63678
Keine Überlappung bei: 63679
Keine Überlappung bei: 63680
Keine Überlappung bei: 63681
Keine Überlappung bei: 63682
Keine Überlappung bei: 63683
Keine Überlappung bei: 63684
Keine Überlappung bei: 63685
Keine Überlappung bei: 63686
Keine Überlappung bei: 63687
Keine Überlappung bei: 63688
Keine Überlappung bei: 63689
Keine Überlappung bei: 63690
Keine Überlappung bei: 63691
Keine Überlappung bei: 63692
Keine Überlappung bei: 63693
Keine Überlappung bei: 63694
Keine Überlappung bei: 63695
Keine Überlappung bei: 63696
Keine Überlappung bei: 63697
Keine Überlapp

Keine Überlappung bei: 63948
Keine Überlappung bei: 63949
Keine Überlappung bei: 63950
Keine Überlappung bei: 63951
Keine Überlappung bei: 63952
Keine Überlappung bei: 63953
Keine Überlappung bei: 63954
Keine Überlappung bei: 63955
Keine Überlappung bei: 63956
Keine Überlappung bei: 63957
Keine Überlappung bei: 63958
Keine Überlappung bei: 63959
Keine Überlappung bei: 63960
Keine Überlappung bei: 63961
Keine Überlappung bei: 63962
Keine Überlappung bei: 63963
Keine Überlappung bei: 63964
Keine Überlappung bei: 63965
Keine Überlappung bei: 63966
Keine Überlappung bei: 63967
Keine Überlappung bei: 63968
Keine Überlappung bei: 63969
Keine Überlappung bei: 63970
Keine Überlappung bei: 63971
Keine Überlappung bei: 63972
Keine Überlappung bei: 63973
Keine Überlappung bei: 63974
Keine Überlappung bei: 63975
Keine Überlappung bei: 63976
Keine Überlappung bei: 63977
Keine Überlappung bei: 63978
Keine Überlappung bei: 63979
Keine Überlappung bei: 63980
Keine Überlappung bei: 63981
Keine Überlapp

Keine Überlappung bei: 64249
Keine Überlappung bei: 64250
Keine Überlappung bei: 64251
Keine Überlappung bei: 64252
Keine Überlappung bei: 64253
Keine Überlappung bei: 64254
Keine Überlappung bei: 64255
Keine Überlappung bei: 64256
Keine Überlappung bei: 64257
Keine Überlappung bei: 64258
Keine Überlappung bei: 64259
Keine Überlappung bei: 64260
Keine Überlappung bei: 64261
Keine Überlappung bei: 64262
Keine Überlappung bei: 64263
Keine Überlappung bei: 64264
Keine Überlappung bei: 64265
Keine Überlappung bei: 64266
Keine Überlappung bei: 64267
Keine Überlappung bei: 64268
Keine Überlappung bei: 64269
Keine Überlappung bei: 64270
Keine Überlappung bei: 64271
Keine Überlappung bei: 64272
Keine Überlappung bei: 64273
Keine Überlappung bei: 64274
Keine Überlappung bei: 64275
Keine Überlappung bei: 64276
Keine Überlappung bei: 64277
Keine Überlappung bei: 64278
Keine Überlappung bei: 64279
Keine Überlappung bei: 64280
Keine Überlappung bei: 64281
Keine Überlappung bei: 64282
Keine Überlapp

In [7]:
# Nur Text und Label werden in 'compare' gespeichert:
d = {'pred_new': twit_comp_new, 'text_new': twit_text_new, 'pred_old': twit_comp_old, 'text_old': twit_text_old}
compare = pd.DataFrame(d)
pickle.dump(compare, open(Path + 'compare_old_new_label.p', "wb" ))

# Speichere DataFrame als csv in dem NUR die nicht gleichen predictions aus old und new stehen:
# compare_no_match = compare[compare['pred_new'] != compare['pred_old']][['text_old', 'pred_old', 'pred_new']]
# compare_no_match.to_csv('/home/lisa/Darmstadt/Master Arbeit/06_Analyse/more_train.csv',sep='\t', header = False, index = False)

# Speichere DataFrame als csv in dem NUR die OFFENSE predictions aus old und new stehen:
compare_OFFENSE = compare.loc[(compare['pred_new'] == 'OFFENSE') &
                               (compare['pred_old'] == 'OFFENSE')][['text_old', 'pred_old', 'pred_new']]
compare_no_match.to_csv('/home/lisa/Darmstadt/Master Arbeit/06_Analyse/more_train_OFFENSE.csv',sep='\t', header = False, index = False)

# Speichere DataFrame als csv in dem NUR die OTHER predictions aus old und new stehen:
compare_OTHER = compare.loc[(compare['pred_new'] == 'OTHER') &
                               (compare['pred_old'] == 'OTHER')][['text_old', 'pred_old', 'pred_new']]
compare_no_match.to_csv(Path + 'more_train_OTHER.csv',sep='\t', header = False, index = False)

In [37]:
import random
# Speichere DataFrame als csv in dem NUR die OFFENSE predictions aus old und new stehen:
compare_OTHER = compare.loc[(compare['pred_new'] == 'OTHER') &
                               (compare['pred_old'] == 'OTHER')][['text_old', 'pred_old', 'pred_new']]

compare_OTHER_sample = random.sample(list(compare_OTHER.index), 1000)
comp_OTHER_samp = compare_OTHER.loc[compare_OTHER_sample]
comp_OTHER_samp.to_csv(Path + 'comp_OTHER_samp.csv',sep='\t', header = False, index = False)

compare_OFFENSE = compare.loc[(compare['pred_new'] == 'OFFENSE') &
                               (compare['pred_old'] == 'OFFENSE')][['text_old', 'pred_old', 'pred_new']]

compare_OFFENSE_sample = random.sample(list(compare_OFFENSE.index), 100)
comp_OFFENSE_samp = compare_OFFENSE.loc[compare_OFFENSE_sample]
comp_OFFENSE_samp.to_csv(Path + 'comp_OFFENSE_samp.csv',sep='\t', header = False, index = False)

# 8540 (GermEval)
# 708 (No_match)
# 100 (OFFENSE) -> nicht nur OFFENSE - Korrektur von ca 10 Tweets
# 994 (OTHER) -> nicht nur OTHER 

## Lade neu-erstellten Gesamtdatensatz und teile ihn in Test und Train Satz auf

In [16]:
from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken
train_imp = get_train_data(Path + 'germeval2018.train_imp.txt')
train = {'Text': train_imp[0], 'Label': train_imp[1], 'other_label': train_imp[2]} 
df_more_train = pd.DataFrame(train)

In [19]:
import random
df_imp_train_rand = random.sample(list(df_more_train.index), 3000)

## TEST ## (3000)
test_rand = df_more_train.loc[df_imp_train_rand]
test_rand.to_csv(Path + 'germEval_test_rand.txt',sep='\t', header = False, index = False)

## TRAIN ## (7343)
more_train = df_more_train.loc[~df_more_train.index.isin(df_imp_train_rand)]
more_train.to_csv(Path + 'germEvalmore_train.txt',sep='\t', header = False, index = False)

In [2]:
import pandas as pd
# Ealuierung muss vergleichbar sein -> deshalb zweiter Versuch:

Eval_Path = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Evaluierung/'

# Train1 ist GermEval Test+Train Set (8541 Zeilen)
# Train2 ist GermEval Test+Train + random Sample von 1000 aus Eval_Test_New (1802 Zeilen)
# Test bei beiden Eval_Test_New (1802 Zeilen) - 1000 random sample

##### 1.) Erstelle Random Sample
from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken
Eval_New_Label = get_train_data(Eval_Path + 'Eval_Test_new_1802.txt')
data = {'Text': Eval_New_Label[0], 'Label': Eval_New_Label[1], 'other_label': Eval_New_Label[2]} 
df_more_data = pd.DataFrame(data)

In [9]:
import random
df_imp_data_rand = random.sample(list(df_more_data.index), 1000) # Random sample index

## Mehr Training ## (1000)
data_rand = df_more_data.loc[df_imp_data_rand] # Zu Index gehörige Daten - random
data_rand.to_csv(Eval_Path + 'self_label_train_1000.txt',sep='\t', header = False, index = False)

## Test ## (802)
test = df_more_data.loc[~df_more_data.index.isin(df_imp_data_rand)]
test.to_csv(Eval_Path + 'self_label_test_802.txt',sep='\t', header = False, index = False)

## Accuracy,  Prediction, Recall, F1

#### Stufen der Klassifizierung:
* Ganz am Anfang nur mit dem Trainingsdatensatz von Germ Eval (Bereits erste Graischen Analysen usw)
* Dann mit dem Tranings UND Test Datensatz von GermEval (Einige Klassifiziert - mit dem Ziel Verbesserung zu schaffen)
* Zum Evaluieren - Training mit random select Trainingsdatensatz
* Final: Trainiere mit allen gelabelten Daten

In [41]:
# Evaluiere mit random train/test Kombbi
# Ergebnisse vermutlich beser als bei Test NUR mit mdp Tweets weil viele der GermEval Tweets in Train und Test Set sind
test_imp = pickle.load(open(Path + 'pickle_TU_Wien/test_imp.p', "rb" )) 
test_imp.columns
testy = test_imp['Label']
yhat_classes = test_imp['predict']

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)

#precision tp / (tp + fp)
precision_OF = precision_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
precision_OT = precision_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Precision: %f' % precision_OF)
print('Precision: %f' % precision_OT)

# recall: tp / (tp + fn)
recall_OF = recall_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
recall_OT = recall_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Recall: %f' % recall_OF)
print('Recall: %f' % recall_OT)

# f1: 2 tp / (2 tp + fp + fn)
f1_OF = f1_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
f1_OT = f1_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('F1 score: %f' % f1_OF)
print('F1 score: %f' % f1_OT)

Accuracy: 0.806333
Precision: 0.674667
Precision: 0.885333
Recall: 0.779261
Recall: 0.819348
F1 score: 0.723202
F1 score: 0.851064


In [44]:
# Trainiert mit ganzem GermEval Set
# Evaluiert mit allen händisch korrigierten Tweets
# Vmtl. schlechter als jetzt 1.) Weil nachtrainiert mit Politiker Tweets
#                            2.) Weil ca 700 Tweets genau die schwer zu klassifizierenden Tweets waren

from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken
Eval_Test_new = get_train_data(Path + 'Eval_Test_new.txt')
test_new = {'Text': Eval_Test_new[0], 'y': Eval_Test_new[1], 'y_hat': Eval_Test_new[2]} 
df_test_new = pd.DataFrame(test_new)

testy = df_test_new['y']
yhat_classes = df_test_new['y_hat']

In [52]:
#accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)

#precision tp / (tp + fp)
precision_OF = precision_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
precision_OT = precision_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Precision: %f' % precision_OF)
print('Precision: %f' % precision_OT)

# recall: tp / (tp + fn)
recall_OF = recall_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
recall_OT = recall_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Recall: %f' % recall_OF)
print('Recall: %f' % recall_OT)

# f1: 2 tp / (2 tp + fp + fn)
f1_OF = f1_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
f1_OT = f1_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('F1 score: %f' % f1_OF)
print('F1 score: %f' % f1_OT)

Accuracy: 0.735294
Precision: 0.581395
Precision: 0.783528
Recall: 0.457038
Recall: 0.856574
F1 score: 0.511771
F1 score: 0.818424


In [ ]:
import pickle, gensim, numpy as np

# Trainiert mit ganzem GermEval Set
# Evaluiert mit 802 händisch korrigierten Tweets

test_imp = pickle.load(open(Eval_Path + 'Eval_1.p', "rb" )) 
test_imp.columns
testy = test_imp['Label']
yhat_classes = test_imp['predict']


#Accuracy: 0.746883
#Precision_OF: 0.606061
#Precision_OT: 0.793046
#Recall_OF: 0.489796
#Recall_OT: 0.859964
#F1 score_OF: 0.541761
#F1 score_OT: 0.825151

In [ ]:
# Trainiert mit ganzem GermEval Set + 1000 selbst gelabelten
# Evaluiert mit 802 händisch korrigierten Tweets

test_imp = pickle.load(open(Eval_Path + 'Eval_2.p', "rb" )) 
test_imp.columns
testy = test_imp['Label']
yhat_classes = test_imp['predict']

#Accuracy: 0.807980
#Precision_OF: 0.693617
#Precision_OT: 0.855379
#Recall_OF: 0.665306
#Recall_OT: 0.870736
#F1 score_OF: 0.679167
F1 score_OT: 0.862989

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)

#precision tp / (tp + fp)
precision_OF = precision_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
precision_OT = precision_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Precision_OF: %f' % precision_OF)
print('Precision_OT: %f' % precision_OT)

# recall: tp / (tp + fn)
recall_OF = recall_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
recall_OT = recall_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('Recall_OF: %f' % recall_OF)
print('Recall_OT: %f' % recall_OT)

# f1: 2 tp / (2 tp + fp + fn)
f1_OF = f1_score(testy, yhat_classes, average="binary", pos_label="OFFENSE")
f1_OT = f1_score(testy, yhat_classes, average="binary", pos_label="OTHER")
print('F1 score_OF: %f' % f1_OF)
print('F1 score_OT: %f' % f1_OT)